In [1]:
import ee
import eemont  # noqa: F401
import geemap
import geemap.colormaps as cm
import pandas as pd

In [2]:
# ee.Authenticate()
ee.Initialize()

In [3]:
buffalo_expansion = ee.FeatureCollection(
    "projects/bison-lab/assets/Buffalo_Expansion_polygons"
)

In [4]:
parcels = buffalo_expansion.filter(
    ee.Filter.inList(
        "Name",
        [
            "Shoshone Tribe",
            "Buffalo Initiative",
            "Hellyer Tribal Lease",
            "Adels Property",
            "Hoopengarner Property",
            "Hellyer Place",
        ],
    )
)

In [5]:
m = geemap.Map()
m.addLayer(parcels, {"color": "red"}, "Parcels")
m.centerObject(parcels, 14)
m

Map(center=[43.20344242266163, -108.80591236968854], controls=(WidgetControl(options=['position', 'transparent…

In [6]:
veg = ee.ImageCollection("projects/rangeland-analysis-platform/vegetation-cover-v3")
    # .closest("2018-01-01")
    # .first()


In [7]:
m = geemap.Map()

m.addLayer(veg, {}, '')

vis_parcels = {
    "color": "000000",
    "width": 4,
    "lineType": "solid",
    "fillColor": "00000000",
}
m.addLayer(parcels.style(**vis_parcels), {}, "Parcels")

m.centerObject(parcels, 14)
m

Map(center=[43.20344242266163, -108.80591236968854], controls=(WidgetControl(options=['position', 'transparent…

### Calculate the mean NDVI for each parcel

In [ ]:
# Calc mean within each parcel
# NOTE: reduceRegions only works on Images, not featureCollections
stats = img.reduceRegions(parcels, "mean", 250)

`stats` is a `FeatureCollection` so let's convert it to a Pandas DataFrame

In [ ]:
df = geemap.ee_to_pandas(stats)
df = df[["Name", "mean"]]

In [ ]:
df

## Time Series

In [ ]:
# Load time series of NDVI
ds = (
    ee.ImageCollection("MODIS/061/MOD13Q1")
    .filterDate("2010-01-01", "2012-01-01")
    .scaleAndOffset()
    .select("NDVI")
)

#### Calc mean within each parcel

In [ ]:
# NOTE: toBands() Converts the image collection to a single multi-band image, necessary to allow reduceRegions
stats = ds.toBands().reduceRegions(parcels, "mean", 250)

`stats` is a `FeatureCollection` so let's convert it to a `pd.DataFrame`

In [ ]:
df = geemap.ee_to_df(stats)

In [ ]:
# Drop un-needed columns
df = df.drop(
    columns=[
        "visibility",
        "descriptio",
        "tessellate",
        "extrude",
        "icon",
        "end",
        "begin",
        "timestamp",
        "altitudeMo",
    ]
)

# Set Names as index and transpose dataframe to have timeseries as index
df = df.set_index("Name").T

# Replace index of strings with datetimes from image collection
dates = pd.DatetimeIndex(
    pd.datetime.fromisoformat(x) for x in geemap.image_dates(ds).getInfo()
)
df.index = pd.DatetimeIndex(dates)

In [ ]:
# Now we have a dataframe with NDVI for each parcel
df

In [ ]:
df.plot(figsize=(12, 6), xlabel="Date", ylabel="NDVI").legend(loc="upper left");